Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('dataset/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df = df.dropna()

In [5]:
X = df.drop('label', axis=1)

In [6]:
y = df['label']

In [7]:
X.shape

(18285, 4)

In [8]:
y.shape

(18285,)

In [9]:
import tensorflow as tf

In [10]:
tf.__version__

'2.4.1'

In [11]:
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [12]:
# Vocabolary Size
voc_size = 5000

In [13]:
messages = X.copy()

In [14]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [15]:
messages.reset_index(inplace=True)

In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\souha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Data Preprocessing

In [18]:
from nltk.stem.porter import PorterStemmer
ps =PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    # print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    # print(review)
    review = ' '.join(review)
    corpus.append(review)

In [19]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [20]:
onehot_repre = [one_hot(words, voc_size) for words in corpus]

In [21]:
onehot_repre[0]

[2548, 3243, 4594, 4263, 3131, 383, 4990, 735, 4321, 2582]

## Embedding Representaion

In [33]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repre, maxlen=sent_length)
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2548,
       3243, 4594, 4263, 3131,  383, 4990,  735, 4321, 2582])

In [34]:
# embedded_vector_features = 100
# model = Sequential()
# model.add(Embedding(voc_size, embedded_vector_features, input_length = sent_length))
# model.add(LSTM(100))
# model.add(Dense(1, activation = 'sigmoid'))
# model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# model.summary()

In [68]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 40)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [50]:

len(embedded_docs),y.shape

(18285, (18285,))

In [51]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [52]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [54]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 8s 26ms/step - loss: 0.4903 - accuracy: 0.7546 - val_loss: 0.2024 - val_accuracy: 0.9118
Epoch 2/10
192/192 [==============================] - 4s 22ms/step - loss: 0.1513 - accuracy: 0.9398 - val_loss: 0.1991 - val_accuracy: 0.9193
Epoch 3/10
192/192 [==============================] - 4s 22ms/step - loss: 0.1075 - accuracy: 0.9616 - val_loss: 0.1992 - val_accuracy: 0.9190
Epoch 4/10
192/192 [==============================] - 5s 25ms/step - loss: 0.0771 - accuracy: 0.9751 - val_loss: 0.2482 - val_accuracy: 0.9171
Epoch 5/10
192/192 [==============================] - 5s 23ms/step - loss: 0.0571 - accuracy: 0.9809 - val_loss: 0.2952 - val_accuracy: 0.9158
Epoch 6/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0364 - accuracy: 0.9879 - val_loss: 0.3834 - val_accuracy: 0.9148
Epoch 7/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0316 - accuracy: 0.9889 - val_loss: 0.3484 - val_accuracy: 0.9150

In [65]:
y_pred=(model.predict(X_test)  > 0.5).astype("int32")

In [66]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3126,  293],
       [ 265, 2351]], dtype=int64)

In [67]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9075393537696769